# Combined

In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest
import os
import matplotlib.font_manager as font_manager
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import statsmodels.stats.multicomp
import matplotlib.dates as mdates
from pylab import figure, text, scatter, show

stats_var = 'Percent_Volvox'

data_path = r"../Data/Figure7_Memory"
data_dir = os.path.abspath(data_path) + '//'

save_path = r"../Code/Graphs"
save_dir = os.path.abspath(save_path) + '//'

if not os.path.exists(save_dir):
    # Create the directory if it does not exist
    os.makedirs(save_dir)

saveplots = True
printstats = False

core_experiment = 'Volvox Memory'
experiment = 'NoLight vs. Continuous Light - ' + core_experiment

title = '$\mathit{Volvox}$ $\mathit{aureus}$ Phototactic Bias\n Persistance Towards Absent Stimulus'

include_mid_sections = False


figurefont = {'fontname': 'Candara',
              'size': 18}

font = font_manager.FontProperties(family='Candara', math_fontfamily='custom', size=12)


mypalette = {"No Light": "saddlebrown", "Continuous": "blue", "No Light (Mock)": "xkcd:light brown"}

orderlist = ['No Light vs. No Light (Mock)', 'Light vs. No Light']

# Load the CSV file
data = pd.read_csv(data_dir+r'Volvox_Aureus_Memory.csv')


# Define font styles
titlefont = {'fontname': 'Candara', 'size': 18}
figurefont = {'fontname': 'Candara', 'size': 16}
tickfont = {'fontname': 'Candara', 'size': 14}
font = font_manager.FontProperties(family='Candara', math_fontfamily='custom', size=12)


def plot_position_bar_chart_and_ttest(data, i, orderlist, title, cmap, xlabel, save_dir, fig, ax, printstats):
    overall_mean = data['avg_x_percent'].mean()
    overall_sem = data['avg_x_percent'].sem()
    if printstats:
        print('overall mean = ', overall_mean, 'overall standard error of the mean = ', overall_sem)

    # Calculate the color based on the mean value
    color = cmap(overall_mean / 100)  # Normalize to [0, 1] range

    # Plot the bar
    bar_height = overall_mean - 50  # Deviation from center
    bar_height_star = overall_mean  # Deviation from center
    bar = ax.bar(i, bar_height, yerr=overall_sem, color=color, edgecolor='black', capsize=10, width=0.3, bottom=50)

    #     ## for 4:
    #     ##pos_list=[0.1275,0.405,0.6825,0.96]
    #     start = 0.1275
    #     end = 0.96

    #     ## for all 5:
    #     # start = 0.12
    #     # end = 0.96

    #     pos_list=np.linspace(start, end, len(orderlist))
    #     x_position=pos_list[i]
    #     print(i, x_position)

    # Add sample size
    sample_size = len(data)
    #     ax.text(x_position, 0.05, f'n={sample_size}', transform=ax.transAxes, **figurefont,
    #             verticalalignment='bottom', horizontalalignment='right')

    plt.annotate(f'n={sample_size}',  # Text to display
                 xy=(i, 30),  # Position at the data point
                 xytext=(0, 10),  # Offset the text 10 points above the data point
                 textcoords='offset points',  # Use offset points for positioning
                 ha='center', **figurefont)

    # Perform two-tailed t-test
    t_stat, p_value = stats.ttest_1samp(data['avg_x_percent'], 50)

    # Calculate Cohen's d
    cohens_d = (overall_mean - 50) / data['avg_x_percent'].std()

    # Add significance stars based on two-tailed p-value
    if p_value < 0.001:
        significance = '***'
    elif p_value < 0.01:
        significance = '**'
    elif p_value < 0.05:
        significance = '*'
    else:
        significance = 'ns'

    if bar_height > -3:
        ax.text(i, bar_height_star + overall_sem + 2, significance, ha='center', va='bottom', **figurefont)
    else:
        ax.text(i, 50 + 2, significance, ha='center', va='bottom',
                **figurefont)
    if printstats:
        print(f"\nStatistics for {title}")
        print(f"Overall average X position (% of chamber width): {overall_mean:.2f}%")
        print(f"Deviation from center: {bar_height:.2f}%")
        print(f"Standard error: {overall_sem:.2f}%")
        print(f"Number of particles: {len(data)}")
        print(f"Two-tailed t-test results:")
        print(f"t-statistic: {t_stat:.4f}")
        print(f"p-value (two-tailed): {p_value:.6f}")
        if p_value < 0.001:
            print("The result is extremely significant (p < 0.001) ***")
        elif p_value < 0.01:
            print("The result is very significant (p < 0.01) **")
        elif p_value < 0.05:
            print("The result is significant (p < 0.05) *")
        else:
            print("The result is not statistically significant (p >= 0.05)")
        print(f"Cohen's d: {cohens_d:.4f}")

    return fig


# Add the font file
font_path = r"../Code/candara-font-family/Candara.ttf"
font_path_full = os.path.abspath(font_path)
font_manager.fontManager.addfont(font_path_full)

plt.rcParams[
    'mathtext.fontset'] = 'custom'  # supported values are ['dejavusans', 'dejavuserif', 'cm', 'stix', 'stixsans', 'custom']
plt.rcParams['mathtext.it'] = 'Candara:italic'
# plt.rcParams['mathtext.fontsize'] = 12

fig, ax = plt.subplots(figsize=(8, 6))

ax.set_title(title, **titlefont)
ax.set_ylabel('Average X Position (% of chamber width)', **figurefont)

ax.set_ylim(30, 70)
# ax.set_xlim(-0.5, 0.5)
# xtick_pos=ax.get_xticks()

# ax.set_xticks([0,1,2,3])
# ax.set_ticklabels(orderlist[0:4])
updated_list = [item.replace('vs ', 'vs. ') for item in orderlist]
plt.xticks(ticks=[0, 1], labels=updated_list[0:2])

ax.set_xlabel('Light Stimulation Pairings', **figurefont)

ax.axhline(y=50, color='black', linestyle='--', linewidth=0.8)

# plt.title(title, **titlefont)
# plt.ylabel('Average Volvox per side (in %)', **figurefont)
# plt.xlabel('1 Hz vs Random', **figurefont)

# Apply font to tick labels
ax.tick_params(axis='both', which='major', labelsize=tickfont['size'], labelfontfamily=tickfont['fontname'])

# Create a colorbar
cmap_memory = LinearSegmentedColormap.from_list('custom', ['saddlebrown', 'white', 'blue'], N=100)
sm = plt.cm.ScalarMappable(cmap=cmap_memory, norm=plt.Normalize(vmin=30, vmax=70))
sm.set_array([])

# Adjust the position of the main axes to make room for the colorbar
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.9, box.height])

# Add the colorbar
cbar = plt.colorbar(sm, ax=ax, orientation='vertical', pad=0.1)
cbar.set_ticks([30, 40, 50, 60, 70])

# cbar.set_ticklabels('High Frequency Side', '', 'Center', '', 'Low Frequency Side')
cbar.set_ticklabels(['No Light Side', '', 'Center', '', 'Light/Mock Side'])
cbar.ax.tick_params(labelsize=tickfont['size'], labelfontfamily=tickfont['fontname'])

for i, x_val in enumerate(orderlist):

    data_i = data.copy()

    data_i = data_i[data_i['Experiment'] == x_val]

    split_xval = x_val.split(' vs. ')
    if split_xval[0]== 'Light':
        f1 = 'Continuous'
    else:
        f1 = split_xval[0]

    if split_xval[1]== 'Light':
        f2 = 'Continuous'
    else:
        f2 = split_xval[1]

    # Define the center positions of each segment
    segment_positions = {
        f1: 1.5 / 2,  # Center of the first 1.5 cm
        f2: 8.5 + 1.5 / 2  # Center of the last 1.5 cm
    }


    # Function to calculate weighted average
    # def weighted_avg(group):
    #     return np.average(group['avg_x_position'], weights=group['Volvox_n'])

    def weighted_avg(group):
        weight_sum = group['Volvox_n'].sum()
        if weight_sum == 0:
            # Return NaN or some other default value if the sum of weights is zero
            return np.nan
        else:
            return np.average(group['avg_x_position'], weights=group['Volvox_n'])


    # Add a column for average x position based on segment
    data_i['avg_x_position'] = data_i['Light Condition'].map(segment_positions)

    # Group by experiment and calculate weighted average x position
    # data_i['exp_id'] = data_i.groupby(['Date', 'Time']).ngroup()

    # Base repeating list
    base_list = [1, 2, 3, 4, 5, 6, 7]

    # Calculate the number of full repeats needed
    num_full_repeats = len(data_i) // len(base_list)
    remainder = len(data_i) % len(base_list)

    # Create the repeating list
    repeating_list = base_list * num_full_repeats + base_list[:remainder]

    # Adjust the list by adding 7 every four/two full iterations
    adjusted_list = []
    for j in range(num_full_repeats):
        if include_mid_sections:
            offset = 7 * (j // 4)
        else:
            offset = 7 * (j // 2)
        adjusted_list.extend([x + offset for x in base_list])

    # Add the remainder elements if necessary
    if remainder > 0:
        if include_mid_sections:
            offset = 7 * (num_full_repeats // 4)
        else:
            offset = 7 * (num_full_repeats // 2)
        adjusted_list.extend([x + offset for x in base_list[:remainder]])

    data_i['exp_id'] = adjusted_list


    weighted_positions = data_i.groupby('exp_id').apply(weighted_avg)

    # Create a new data_iFrame to store the results
    summary_df = pd.DataFrame({
        'exp_id': weighted_positions.index,
        'weighted_avg_x_position': weighted_positions
    })

    summary_df= summary_df.dropna()

    # Calculate the average x position as a percentage of the chamber length
    summary_df['avg_x_percent'] = (1 - summary_df['weighted_avg_x_position'] / 10) * 100
    # Verify the result
    # print(summary_df['avg_x_percent'])
    # Plot and perform tests for '1Hz vs Random'

    fig = plot_position_bar_chart_and_ttest(summary_df, i, orderlist, title, cmap_memory, x_val, save_dir, fig, ax,
                                            printstats)

    plt.tight_layout()

    if saveplots:
        fig.savefig(save_dir + 'Volvox_Aureus_Phototactic_Memory.png', dpi=300,
                    bbox_inches='tight')
    else:
        plt.show()



